# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back', 'rb'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [79]:
# %pylab inline
%matplotlib inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

In [80]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print (classes)

['Back', 'Belly', 'Crouch', 'Frog', 'HeadBack', 'Knee', 'Left', 'Right', 'Sit', 'Stand', 'StandInit']


In [81]:
class ExtractStrToBinaries:
    def __init__(self, file):
        self.file = file

    def read(self, length):
        return self.file.read(length).encode()

    def readline(self):
        return self.file.readline(-1).encode()


def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE

    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(ExtractStrToBinaries(open(filename, 'r')))

    target = [i] * len(data)
    return data, target

In [82]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE

for i in range(0, len(classes)):
    data, target = load_pose_data(i)
    all_data += data
    all_target += target

all_data = np.asarray(all_data)
all_target = np.asarray(all_target)
print('total number of data:', len(all_data))

total number of data: 222


In [83]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [84]:
clf = svm.SVC(gamma=0.001, C=100)

### learning

In [85]:
arr = np.asarray([1, 2, 3, 4, 5, 6, 7, 8, 9])
sub_arr = np.asarray([1, 2, 3, 4, 5])


print(arr[~np.in1d(arr, sub_arr)])


[6 7 8 9]


In [86]:
# YOUR CODE HERE
clf.fit(all_data[training_data], all_target[training_data])

SVC(C=100, gamma=0.001)

### predicting

In [87]:
clf.predict(all_data[-1:]), all_target[-1:]

(array([10]), array([10]))

In [88]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [89]:
expected = []
predicted = []
# YOUR CODE HERE

expected = all_target[training_data]
predicted = clf.predict(all_data[training_data])

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       1.00      1.00      1.00        13
           2       0.96      1.00      0.98        23
           3       1.00      1.00      1.00         8
           4       1.00      1.00      1.00         7
           5       1.00      1.00      1.00         8
           6       1.00      1.00      1.00        13
           7       1.00      0.86      0.92         7
           8       1.00      1.00      1.00        21
           9       1.00      1.00      1.00         7
          10       1.00      1.00      1.00        33

    accuracy                           0.99       155
   macro avg       1.00      0.99      0.99       155
weighted avg       0.99      0.99      0.99       155


Confusion matrix:
[[15  0  0  0  0  0  0  0  0  0  0]
 [ 0 13  0  0  0  0  0  0  0  0  0]
 [ 0  0 23  0  0  0  0  0  0  0  0]
 [ 0  0  0  8  0  0  0  0  0  0  0]
 

## 4. Evaluate on the test data

In [90]:
expected = []
predicted = []

# YOUR CODE HERE
expected = all_target[-n_training_data:]
predicted = clf.predict(all_data[-n_training_data:])

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           2       0.83      1.00      0.91         5
           3       1.00      1.00      1.00        10
           4       1.00      1.00      1.00        10
           5       1.00      1.00      1.00        10
           6       1.00      1.00      1.00        20
           7       1.00      0.91      0.95        11
           8       1.00      1.00      1.00        26
           9       1.00      1.00      1.00        11
          10       1.00      1.00      1.00        52

    accuracy                           0.99       155
   macro avg       0.98      0.99      0.98       155
weighted avg       0.99      0.99      0.99       155


Confusion matrix:
[[ 5  0  0  0  0  0  0  0  0]
 [ 0 10  0  0  0  0  0  0  0]
 [ 0  0 10  0  0  0  0  0  0]
 [ 0  0  0 10  0  0  0  0  0]
 [ 0  0  0  0 20  0  0  0  0]
 [ 1  0  0  0  0 10  0  0  0]
 [ 0  0  0  0  0  0 26  0  0]
 [ 0  0  0  0  0  0  0 11  0]
 [ 0  0  0  0

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [91]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'wb'))

Then, in the application we can load the trained classifier again.

In [92]:
clf2 = pickle.load(open(ROBOT_POSE_CLF, 'rb' ))
clf2.predict(all_data[-1:]), all_target[-1:]

(array([10]), array([10]))